## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [2]:
model_name = "waktaverse-gemma-ko-7b-it" # ADD YOUR MODEL NAME HERE
username = "PathFinderKR" # ADD YOUR USERNAME HERE

In [3]:
# hub
repo_id = f"{username}/{model_name}" # repository id

## Imports

In [4]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import PeftModel

## Device

In [5]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# solar variants
# "chihoonlee10/T3Q-ko-solar-dpo-v4.0"

In [7]:
base_model_id = "google/gemma-7b-it"

## Store Peft Weights

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Push to Hub

In [ ]:
model.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)